In [1]:
#!/usr/bin/env python
# coding: utf-8

"""
Test code written by Viresh Ranjan

Last modified by: Minh Hoai Nguyen (minhhoai@cs.stonybrook.edu)
Date: 2021/04/19
"""

import copy
from model import  Resnet50FPN,Wide_Resnet50_2,VGG16FPN,CountRegressor,weights_normal_init
from utils_ltce import MAPS, Scales, Transform, extract_features
from utils_ltce import MincountLoss, PerturbationLoss
from PIL import Image
import os
import torch
import argparse
import json
import numpy as np
from tqdm import tqdm
from os.path import exists
import torch.optim as optim
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
from collections import defaultdict
import torchvision.ops.boxes as bops

In [2]:
#data_path = '/Users/alessandroquattrociocchi/Git/AML/Final_Project/data/'
data_path = '/Users/alessandroquattrociocchi/Documents/Courses /2/2.1/AML/homeworks/FinalProject/Final_Project/data/'
output_dir = "./logsSave"
test_split = "test" #choices=["train", "test", "val"]
gpu_id = 0 
learning_rate = 1e-5
data_path = data_path
anno_file = data_path + 'annotation_FSC147_384.json'
data_split_file = data_path + 'Train_Test_Val_FSC_147.json'
im_dir = data_path + 'images_384_VarV2'
gt_dir = data_path + 'gt_density_map_adaptive_384_VarV2'
pre_trained_backbone = 'resnet' #choices=[resnet,wide_resnet,vgg16]
model_path = data_path + "/pretrainedModels/FamNet_Save1.pth"

In [3]:
from collections import Counter

model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5l6', pretrained=True)

def count_class(results):

    dict_ = defaultdict(int)
    for i in results:
        dict_[i['name']] += 1
    num = max(dict_.values())

    return num


def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]

def intersects(box1, box2):

    box1 = torch.tensor([box1], dtype=torch.float)
    box2 = torch.tensor([box2], dtype=torch.float)
    iou = bops.box_iou(box1, box2).numpy()[0][0]
    return iou


Using cache found in /Users/alessandroquattrociocchi/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-12-8 torch 1.10.0 CPU

Fusing layers... 
[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.
Model Summary: 476 layers, 76726332 parameters, 0 gradients
Adding AutoShape... 


In [20]:
def YOLO_boxes(results_yolo, annotations, threshold=4):
    """
    E.g. annotations = annotations['6.jpg']['box_examples_coordinates']
    This function takes as input the results of YOLO and returns a set of boxes which will then be used for the
    feature extraction part. In order to do this we will review a few things:
    1. Yolo needs to detect at least 3 or more objects. If less than this number of objects has been detected we remove this observation
    2. We keep only information regarding the most common object (which is typically the object we are trying to identify)
    3. We remove the boxes that are overlapping (we don't want to provide FamNet with two similar boxes)
    4. Keep only the boxes that are over a certain threshold level
        a. if threshold is int take top threshold
        b. if threshold is float take values above threshold
    5. We provide as output the boxes that fulfill all of the previous requirements
    """

    # Remove all classes that don't belong to the most common class
    list_ = []
    for i in results_yolo:
        list_.append(i['name'])
    
    if len(list_) == 0:
        return None

    name = most_frequent(list_)
    final_dict = [i for i in results_yolo if i['name'] == name]

    # More than 3 objects detected by YOLO
    if len(final_dict) < 3:
        return None

    # Check for overlaps 
    non_overlap = list(range(1, len(results_yolo)+1))
    overlap = []
    for example_box in annotations:
        list_original = [example_box[0][0], example_box[0][1], 
                        example_box[2][0], example_box[2][1]]
        for i, yolo_res in enumerate(results_yolo):
            list_yolo = [yolo_res['xmin'], yolo_res['ymin'],
                        yolo_res['xmax'], yolo_res['ymax']]
                        
            if intersects(list_yolo, list_original) <= 0.10:
                overlap.append(i+1)

    non_overlap = list(set(non_overlap).difference(set(overlap)))
    results_yolo = [results_yolo[i-1] for i in non_overlap]

    # Keep only the most probable boxes
    list_ = []
    for i in results_yolo:
        list_.append(i['confidence'])
    order = list(np.argsort(list_)[::-1])
    if isinstance(threshold, int):
        results_yolo = [results_yolo[i] for i in order[:threshold]]
    elif isinstance(threshold, float):
        order_float = list(np.sort(list_)[::-1])
        pos = len([i for i in order_float if i >= threshold])
        results_yolo = [results_yolo[i] for i in order[:pos]]

    # Return boxes as in the annotation file
    final_list = []
    for i in results_yolo:
        final_list.append([i['xmin'], i['ymin'], i['xmax'], i['ymax']])

    return final_list

In [5]:
if not exists(anno_file) or not exists(im_dir):
    print("Make sure you set up the --data-path correctly.")
    print("Current setting is {}, but the image dir and annotation file do not exist.".format(data_path))
    print("Aborting the evaluation")
    exit(-1)

if not torch.cuda.is_available() or gpu_id < 0:
    use_gpu = False
    print("===> Using CPU mode.")
else:
    use_gpu = True
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

===> Using CPU mode.


In [6]:
with open(anno_file) as f:
    annotations = json.load(f)

with open(data_split_file) as f:
    data_split = json.load(f)

In [11]:
def test(data, num_img, pre_trained_backbone, yolo_flag, yolo_threshold, plot_flag=False, im_dir=im_dir, use_gpu=False, annotations=annotations, model_path='model.pth',
         adapt=True, gradient_steps=100, learning_rate=1e-7):

    weight_mincount = 1e-9
    weight_perturbation = 1e-4

    if pre_trained_backbone == 'vgg16':
        backbone = VGG16FPN()
    elif pre_trained_backbone == 'resnet':
        backbone = Resnet50FPN()
    elif pre_trained_backbone == 'wide_resnet':
        backbone = Wide_Resnet50_2()

    if use_gpu: backbone.cuda()
    backbone.eval()

    regressor = CountRegressor(6, pool='mean')
    
    if use_gpu:
        regressor.load_state_dict(torch.load(model_path))
    else:
        regressor.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))

    if use_gpu: regressor.cuda()
    regressor.eval()

    cnt = 0
    SAE = 0  # sum of absolute errors
    SSE = 0  # sum of square errors

    print("Evaluation on {} data".format(test_split))
    n_imgs = num_img
    im_ids = data[:n_imgs]

    pbar = tqdm(im_ids)
    for im_id in pbar:
        anno = annotations[im_id]
        bboxes = anno['box_examples_coordinates']
        dots = np.array(anno['points'])
        image_path = '{}/{}'.format(im_dir, im_id)

        rects = list()
        for bbox in bboxes:
            x1, y1 = bbox[0][0], bbox[0][1]
            x2, y2 = bbox[2][0], bbox[2][1]
            rects.append([y1, x1, y2, x2])

        if yolo_flag:
                
            detections = model_yolo(image_path)
            results_yolo = detections.pandas().xyxy[0].to_dict(orient="records")

            try:
                yolo_obj_cnt = count_class(results_yolo)
            except:
                print("Yolo Failed")
                yolo_obj_cnt=0

            for result in results_yolo:
                con = result['confidence']
                cs = result['class']
                x1 = int(result['xmin'])
                y1 = int(result['ymin'])
                x2 = int(result['xmax'])
                y2 = int(result['ymax'])

            yolo_res = YOLO_boxes(results_yolo, bboxes, threshold=yolo_threshold)

            frame_1 = cv2.imread(image_path)
            frame_1 = cv2.cvtColor(frame_1, cv2.COLOR_BGR2RGB)
            
            frame_2 = cv2.imread(image_path)
            frame_2 = cv2.cvtColor(frame_2, cv2.COLOR_BGR2RGB)

            for i in bboxes:
                x1 = i[0][0]
                y1 = i[0][1]
                x2 = i[2][0]
                y2 = i[2][1]
                # Do whatever you want
                f1 = cv2.rectangle(frame_1, (x1, y1), (x2, y2),color=(255,0,0),thickness=3)
            
            if plot_flag:
                plt.imshow(f1)
                plt.show()
                plt.close()

            print('')

            if yolo_res:
                for i in yolo_res:
                    x1 = int(i[0])
                    y1 = int(i[1])
                    x2 = int(i[2])
                    y2 = int(i[3])
                    # Do whatever you want
                    f2 = cv2.rectangle(frame_2, (x1, y1), (x2, y2),color=(0,255,0),thickness=3)
            
                if plot_flag:
                    plt.imshow(f2)
                    plt.show()
                    plt.close()

            if yolo_res:
                rects += yolo_res

        image = Image.open('{}/{}'.format(im_dir, im_id))
        image_path = '{}/{}'.format(im_dir, im_id)
        image.load()
        image.show()
        sample = {'image': image, 'lines_boxes': rects}
        sample = Transform(sample)
        image, boxes = sample['image'], sample['boxes']

        if use_gpu:
            image = image.cuda()
            boxes = boxes.cuda()

        with torch.no_grad(): features = extract_features(backbone, image.unsqueeze(0), boxes.unsqueeze(0), MAPS, Scales)

        if not adapt:
            with torch.no_grad(): output = regressor(features)
        else:
            features.required_grad = True
            adapted_regressor = copy.deepcopy(regressor)
            adapted_regressor.train()
            optimizer = optim.Adam(adapted_regressor.parameters(), lr=learning_rate)
            for step in range(0, gradient_steps):
                optimizer.zero_grad()
                output = adapted_regressor(features)
                lCount = weight_mincount * MincountLoss(output, boxes)
                lPerturbation = weight_perturbation * PerturbationLoss(output, boxes, sigma=8)
                Loss = lCount + lPerturbation
                # loss can become zero in some cases, where loss is a 0 valued scalar and not a tensor
                # So Perform gradient descent only for non zero cases
                if torch.is_tensor(Loss):
                    Loss.backward()
                    optimizer.step() 
            features.required_grad = False
            output = adapted_regressor(features)


        gt_cnt = dots.shape[0]
        pred_cnt = output.sum().item()
        cnt = cnt + 1
        err = abs(gt_cnt - pred_cnt)
        SAE += err
        SSE += err**2

        pbar.set_description('{:<8}: actual-predicted: {:6d}, {:6.1f}, error: {:6.1f}. Current MAE: {:5.2f}, RMSE: {:5.2f}, YOLO: {:6.1f}'.\
                            format(im_id, gt_cnt, pred_cnt, abs(pred_cnt - gt_cnt), SAE/cnt, (SSE/cnt)**0.5, yolo_obj_cnt))
        print("")

    print('On {} data, MAE: {:6.2f}, RMSE: {:6.2f}'.format(test_split, SAE/cnt, (SSE/cnt)**0.5))



In [12]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
test(data=data_split['test'], num_img=10, pre_trained_backbone='vgg16', yolo_flag=True, yolo_threshold=3, plot_flag=True, im_dir=im_dir, use_gpu=False, annotations=annotations, model_path=model_path,
         adapt=False, gradient_steps=100, learning_rate=1e-7)

In [19]:
!pwd

/Users/alessandroquattrociocchi/Documents/Courses /2/2.1/AML/homeworks/FinalProject/Final_Project
